In [ ]:
%pylab inline
import numpy as np
from sklearn import preprocessing
from keras.datasets import mnist
from keras.utils import np_utils
np.set_printoptions(suppress=True)
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
def preproc(X_train, y_train):
    X = X_train
    X = X.reshape(X.shape[0], X.shape[1] * X.shape[2]).astype("float32")
    #X = X/255
    X = preprocessing.scale(X)
    Y = np_utils.to_categorical(y_train, 10)
    return X, Y

X, Y = preproc(X_train, y_train)
Xt, Yt = preproc(X_test, y_test)

imshow(X[5].reshape((sqrt(X[1].shape),sqrt(X[1].shape))))

In [288]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop

# uniform init
np.random.seed(1337)

# 208896 params
model = Sequential()
"""
model.add(Dense(784, 256))
model.add(Activation('tanh'))
model.add(Dense(256, 128))
model.add(Activation('tanh'))
model.add(Dense(128, 64))
model.add(Activation('tanh'))
model.add(Dense(64, 32))
model.add(Activation('tanh'))
model.add(Dense(32, 64))
model.add(Activation('tanh'))
model.add(Dense(64, 128))
model.add(Activation('tanh'))
model.add(Dense(128, 256))
model.add(Activation('tanh'))
model.add(Dense(256, 784))
"""
model.add(Dense(784, 256))
model.add(Activation('tanh'))
model.add(Dense(256, 32))
model.add(Activation('tanh'))
model.add(Dense(32, 256))
model.add(Activation('tanh'))
model.add(Dense(256, 784))


# optimize
#opt = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
opt = Adam()
model.compile(loss='mean_squared_error', optimizer=opt)

In [289]:
# repro
model.fit(X, X, batch_size=128, nb_epoch=10)

Epoch 0


ValueError: dimension mismatch in args to gemm (128,64)x(256,784)->(128,784)
Apply node that caused the error: GpuGemm{inplace}(GpuFromHost.0, TensorConstant{1.0}, GpuElemwise{Composite{tanh((i0 + i1))}}[(0, 0)].0, <CudaNdarrayType(float32, matrix)>, TensorConstant{-1.0})
Inputs types: [CudaNdarrayType(float32, matrix), TensorType(float32, scalar), CudaNdarrayType(float32, matrix), CudaNdarrayType(float32, matrix), TensorType(float32, scalar)]
Inputs shapes: [(128, 784), (), (128, 64), (256, 784), ()]
Inputs strides: [(784, 1), (), (64, 1), (784, 1), ()]
Inputs values: ['not shown', array(1.0, dtype=float32), 'not shown', 'not shown', array(-1.0, dtype=float32)]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' fo

In [ ]:
img = 5

# 784*32 + 32 + 32*784 + 784 = 50992
#print W.shape, b.shape, Wp.shape, bp.shape
#W,b,Wp,bp = map(lambda x: x.get_value(), model.params)

im = X[img].reshape((sqrt(X[1].shape),sqrt(X[1].shape)))
figure(), imshow(im)

#x = X[img]
#m = np.tanh(np.dot(x, W) + b)
#y = np.dot(m, Wp) + bp
y = model.predict((X[img:img+1]))

pred = y.reshape((sqrt(X[1].shape),sqrt(X[1].shape)))
figure(), imshow(pred)

print np.mean((im-pred)**2)

In [ ]:
model.evaluate(X, X, verbose=2)
model.evaluate(Xt, Xt, verbose=2)
pass

In [ ]:
import pickle

pickle.dump(map(lambda x: x.get_value(), model.params), open("sample/sgdweights", "wb"))

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=32)
pca.fit(X)

In [ ]:
def do_pca(x):
    # pca.mean_ doesn't even matter
    Xt = np.dot(x, pca.components_.T)
    return np.dot(Xt, pca.components_)

Xll = map(do_pca, X)
print np.mean((Xll - X)**2)

In [ ]:
imshow(np.dot(pca.components_.T, pca.components_))